In [1]:
import pandas as pd
from config import load_config
from sklearn.model_selection import train_test_split

In [2]:
cfg = load_config('Kaki-cfg.yml')
# print(cfg)

# Method-01: Manual

In [ ]:
df = pd.read_csv(f'{cfg.path.dataset}/metadata.csv')

In [4]:
l = df.shape[0]
split = df['split'].values
split[0     : l-l//4] = 'train'
split[l-l//4: l-l//8] = 'dev'
split[l-l//8:       ] = 'test'

df['split'] =  split

In [5]:
df.to_csv(f'{cfg.path.dataset}/metadata.csv')

# Method-02: sklearn

In [ ]:
df = pd.read_csv(f'{cfg.path.dataset}/metadata.csv')

In [ ]:
dist_cols_name = [el for el in list(df.keys()) if 'distance' in el]
df['distance'] =  df[dist_cols_name[0]]
for col in dist_cols_name:
    df['distance'].update(df[col])
df['distance'] = df['distance'] * 111

step = 30
for ii in range(0, int(df['distance'].max()), step):
    cond = (ii < df['distance']) &  (df['distance'] <= ii+step)
    df.loc[cond, 'distance'] = ii

# df['distance'].hist(bins=range(0, 200, step), edgecolor='k', log=True)

In [89]:
target_cols = ['source_magnitude',
            #    'source_latitude_deg',
            #    'source_longitude_deg',
            #    'source_depth_km',
               'distance']
#
stratify_keys = []
for key in target_cols:
    new_key = f'{key}_modif'
    df[new_key] = df[key].round(0)
    stratify_keys.append(new_key)
#
for key in stratify_keys:
    col = df[key]
    cond = col.isna()
    df.loc[cond, key] = -1

In [90]:
train, test = train_test_split(
    df, test_size=0.10, random_state=42,
    stratify=df[stratify_keys])
#
df.loc[train.index, 'split'] = 'train'
df.loc[test.index, 'split'] = 'test'

In [91]:
df_train = df[df['split']=='train']
train, dev = train_test_split(
    df_train, test_size=0.10, random_state=42,
    stratify=df_train[stratify_keys])

df.loc[train.index, 'split'] = 'train'
df.loc[dev.index, 'split'] = 'dev'

In [92]:
df.drop(columns=stratify_keys, inplace=True)

In [93]:
df.to_csv(f'{cfg.path.dataset}/metadata.csv')

# TMP